In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [2]:
!pip install pyspark==2.4.5

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [5]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [19]:
import random
rdd = sc.parallelize(random.sample(range(1000),100))
print(rdd.take(10))

[136, 541, 403, 437, 145, 499, 339, 473, 504, 167]


In [14]:
# mean
sum = rdd.sum()
n = rdd.count()
mean = sum/float(n)
print(mean)

509.12


In [38]:
# median
sorted = rdd.sortBy(lambda x: x)
sorted_and_index = sorted.zipWithIndex()
sorted_with_index = sorted_and_index.map(lambda x: (x[1],x[0]))
n = sorted_with_index.count()

if n % 2 == 1:
    index = (n-1)/2
    median = sorted_with_index.lookup(index)[0]
else:
    index1 = n/2 - 1
    index2 = n/2
    median = (sorted_with_index.lookup(index1)[0] +  sorted_with_index.lookup(index2)[0]) / 2

print(median)

487.5


In [39]:
# standard deviation
from math import sqrt
sd = sqrt(rdd.map(lambda x: pow(x-mean,2)).sum()/n)
print(sd)

288.0346888831274


In [41]:
# skewness
n = float(n)
skewness = (rdd.map(lambda x: pow(x-mean,3)).sum()/pow(stdv,3))/n
skewness

-0.23349202874223912

In [43]:
# kurtosis
kurtosis = (rdd.map(lambda x: pow(x-mean,4)).sum()/pow(stdv,4))/n
kurtosis

1.85327962407774

In [48]:
# covariance
rddX = sc.parallelize(random.sample(range(1000),100))
rddY = sc.parallelize(random.sample(range(1000),100))
meanX = rddX.sum()/float(rddX.count())
meanY = rddY.sum()/float(rddY.count())
rddXY = rddX.zip(rddY)
covXY = rddXY.map(lambda x: (x[0] - meanX)*(x[1] - meanY)).sum() / rddXY.count()

# correlation
sdX = sqrt(rddX.map(lambda x: pow(x-meanX,2)).sum()/rddX.count())
sdY = sqrt(rddY.map(lambda x: pow(x-meanY,2)).sum()/rddY.count())
corrXY = covXY/(sdX*sdY)

print(covXY)
print(corrXY)

3663.377599999997
0.047385147192775055


In [55]:
# covariance matrix
from pyspark.mllib.stat import Statistics
col1 = sc.parallelize(random.sample(range(1000),100))
col2 = sc.parallelize(random.sample(range(1000),100))
col3 = sc.parallelize(random.sample(range(1000),100))
col4 = sc.parallelize(random.sample(range(1000),100))
data = col1.zip(col2).zip(col3).zip(col4)
data = data.map(lambda x: [x[0][0][0],x[0][0][1],x[0][1],x[1]])
Statistics.corr(data)

array([[ 1.        ,  0.11566108, -0.22179425,  0.03930211],
       [ 0.11566108,  1.        , -0.0432733 ,  0.02781347],
       [-0.22179425, -0.0432733 ,  1.        ,  0.08601731],
       [ 0.03930211,  0.02781347,  0.08601731,  1.        ]])